In [1]:
import pandas as pd
import numpy as np
import random as rnd
import os
import matplotlib.pyplot as plt
import matplotlib



In [2]:
from sklearn import preprocessing
import datetime

In [3]:
from sklearn.model_selection import train_test_split


In [4]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [5]:
from sklearn.model_selection import GridSearchCV

In [6]:
import xgboost as clf



In [7]:
chunks=pd.read_csv("C://Users/Abhishek Parashar//Desktop//.ipynb_checkpoints//Training Data.csv")
print('done')

done


In [8]:
time_prior = 10
time_duration = 24
time_sep = 24

In [ ]:
dt = {}
colms = ['paO2_FiO2', 'platelets_x_1000',
       'total_bilirubin', 'urinary_creatinine', 'creatinine', 'HCO3', 'pH',
       'paCO2', 'direct_bilirubin', 'excess', 'ast', 'bun', 'calcium',
       'glucose', 'lactate', 'magnesium', 'phosphate', 'potassium', 'hct',
       'hgb', 'ptt', 'wbc', 'fibrinogen', 'troponin', 'GCS_Score', 'heartrate',
       'respiration', 'temperature']
pids = df.patientunitstayid.unique()
for pid in pids:
    for col in colms:
        dt[(pid, col)] = df[df['patientunitstayid']==pid][col].median()
print("dictDone")

In [ ]:

for pid in pids:
    for col in colms:
        df.loc[(df.patientunitstayid==pid) & (df[col].isnull()), col] = dt[(pid, col)]
print('complete')

In [ ]:
df.to_csv('table_filled')

In [9]:
def feature_fun(col, df):
    
    standard_devaition = df[col].std()
    kurtosis = df[col].kurtosis()
    skewness = df[col].skew()
    mean = df[col].mean()
    minimum = df[col].min()
    maximum = df[col].max()
    rms_diff = (sum(df[col].diff().fillna(0, inplace=False).apply(lambda x: x*x))/(len(df)+1))**0.5
    return standard_devaition, kurtosis, skewness, mean, minimum, maximum, rms_diff

In [ ]:
i=1
for chunk in chunks:  
    process(chunk, i)
    print(i)
    i+=1

1
2
3


In [3]:
df = pd.read_csv('Sepsis2-6'+str(1)+'.csv')
for i in range(154):
    df = pd.concat([df, pd.read_csv('Sepsis2-6'+str(i+2)+'.csv')])
df.reset_index(drop=True, inplace=True)

In [4]:
print(sum(df['Y']), len(df))

(11046, 194570)


In [ ]:
df[df['Y']==1]

In [31]:
def cases_df(df):
    df.drop(columns=['Unnamed: 0'], inplace = True)
    df.dropna(inplace=True)
    sepsis_df = df[df['label']==1]
    return sepsis_df

In [32]:
def df_preprocess(df):
    df.dropna(inplace=True)
    sepsis_df = df[df['label']==0]
    return sepsis_df

In [33]:
sepsis_df = cases_df(df)
labels = sepsis_df['Y']
sepsis_X_train, sepsis_x_cv, sepsis_label, sepsis_y_cv = train_test_split(sepsis_df, labels, test_size=0.2, random_state=23)


In [34]:
controls_df = df_preprocess(df)
labels = controls_df['Y']
X_train, x_cv, label, y_cv = train_test_split(controls_df, labels, test_size=0.2, random_state=23)


In [64]:
print(len(sepsis_df), len(controls_df))

(6834, 181494)


In [8]:
def get_controls(df):
    downsampled_df, _, _, _ = train_test_split(df, df['label'], test_size=0.01)
    return downsampled_df

In [42]:
params = {'eta': 0.1, 'max_depth': 6, 'scale_pos_weight': 1, 'objective': 'reg:linear','subsample':0.25,'verbose': False}
xgb_model = None

In [50]:
params = {'eta': 0.1, 'max_depth': 15, 'scale_pos_weight': 10, 'objective': 'reg:linear','subsample':0.25,'verbose': False}
xgb_model = None
pf = pd.concat([sepsis_X_train, X_train])
labels = pf['label']
temp, X_cv, label, Y_cv = train_test_split(pf, labels, test_size=0.2)
xg_train_1 = clf1.DMatrix(temp.drop(['label'],axis=1), label=label)
xg_test = clf1.DMatrix(X_cv.drop(['label'],axis=1), label=Y_cv)
model = clf1.train(params, xg_train_1, 50, xgb_model=xgb_model)
print(classification_report(Y_cv, (model.predict(xg_test)>0.5).astype(int)))
print('F1 score:', f1_score(Y_cv, (model.predict(xg_test)>0.5).astype(int)))

[09:03:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
             precision    recall  f1-score   support

          0       0.99      0.99      0.99     29006
          1       0.74      0.78      0.76      1127

avg / total       0.98      0.98      0.98     30133

('F1 score:', 0.75736568457538989)


In [10]:
import copy

In [68]:
sepsis_X_train.head()

,GCS_Score_kurtosis,GCS_Score_maximum,GCS_Score_mean,GCS_Score_minimum,GCS_Score_rms_diff,GCS_Score_skewness,GCS_Score_std,HCO3_kurtosis,HCO3_maximum,HCO3_mean,...,urinary_creatinine_rms_diff,urinary_creatinine_skewness,urinary_creatinine_std,wbc_kurtosis,wbc_maximum,wbc_mean,wbc_minimum,wbc_rms_diff,wbc_skewness,wbc_std
117554,1.284869,15.0,14.812500,14.0,0.242536,-1.771925,0.403113,0.000000,24.100000,24.100004,...,0.0,0.0,0.0,1.284877,16.200001,16.106249,15.7,0.121268,-1.771928,0.201557
193271,0.000000,14.0,14.000000,14.0,0.000000,0.000000,0.000000,-0.765031,32.299999,25.720001,...,0.0,0.0,0.0,4.999991,13.300000,12.780000,10.7,1.061446,-2.236066,1.162756
184680,5.999999,14.0,4.833333,3.0,4.157609,2.449490,4.490731,-1.875000,24.100000,21.900002,...,0.0,0.0,0.0,0.000000,14.000000,14.000000,14.0,0.000000,0.000000,0.000000
30763,0.000000,11.0,11.000000,11.0,0.000000,0.000000,0.000000,4.355614,24.100000,18.748005,...,0.0,0.0,0.0,9.640832,16.299999,15.851997,10.7,1.098250,-3.297526,1.550570
68782,0.000000,14.0,14.000000,14.0,0.000000,0.000000,0.000000,-0.363622,24.900000,21.833332,...,0.0,0.0,0.0,8.999998,19.600000,8.755556,7.4,3.857979,3.000000,4.066667


In [25]:

runs = 80
Temp_X_cv = copy.copy(sepsis_x_cv)
Temp_y_cv = copy.copy(sepsis_y_cv)
for i in range(runs):
    pf = pd.concat([sepsis_X_train, get_controls(X_train).reset_index(drop=True)])
    labels = pf['Y']
    print("count: ", i+1)
    print(sum(labels), len(labels))
    if True:
        temp, X_cv, label, Y_cv = train_test_split(pf, labels, test_size=0.05)
        xg_train_1 = clf.DMatrix(temp.drop(['Y'],axis=1), label=label)
        xg_test = clf.DMatrix(X_cv.drop(['Y'],axis=1), label=Y_cv)
        model = clf.train(params, xg_train_1, 50, xgb_model=xgb_model)
        print(classification_report(Y_cv, (model.predict(xg_test)>0.5).astype(int)))
        print('F1 score:', f1_score(Y_cv, (model.predict(xg_test)>0.5).astype(int)))
        Temp_X_cv = pd.concat([Temp_X_cv, X_cv])
        Temp_y_cv = pd.concat([Temp_y_cv, Y_cv])
print("1st cv score")
print(classification_report(Temp_y_cv, (model.predict(clf1.DMatrix(Temp_X_cv.drop(['Y'],axis=1), label=Temp_y_cv))>0.5).astype(int)))
print('F1 score:', f1_score(Temp_y_cv, (model.predict(clf1.DMatrix(Temp_X_cv.drop(['Y'],axis=1), label=Temp_y_cv))>0.5).astype(int)))
print("2nd cv score")
CV_X = pd.concat([sepsis_x_cv, x_cv])
cv_y = pd.concat([sepsis_y_cv, y_cv])
print(classification_report(cv_y, (model.predict(clf1.DMatrix(CV_X.drop(['Y'],axis=1), label=cv_y))>0.5).astype(int)))
print('F1 score:', f1_score(cv_y, (model.predict(clf1.DMatrix(CV_X.drop(['Y'],axis=1), label=cv_y))>0.5).astype(int)))


('count: ', 1)
(5467, 149210)
[07:58:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      7204
          1       0.85      0.65      0.74       257

avg / total       0.98      0.98      0.98      7461

('F1 score:', 0.73568281938325986)
('count: ', 2)
(5467, 149210)
[07:58:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      7178
          1       0.93      0.73      0.82       283

avg / total       0.99      0.99      0.99      7461

('F1 score:', 0.81746031746031744)
('count: ', 3)
(5467, 149210)
[07:58:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
             precision    rec

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7180
          1       1.00      0.95      0.97       281

avg / total       1.00      1.00      1.00      7461

('F1 score:', 0.97080291970802901)
('count: ', 22)
(5467, 149210)
[08:01:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7203
          1       1.00      0.94      0.97       258

avg / total       1.00      1.00      1.00      7461

('F1 score:', 0.97005988023952106)
('count: ', 23)
(5467, 149210)
[08:01:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7165
          1       1.00      0.96      0.98       296

avg / total       1.00 

('count: ', 42)
(5467, 149210)
[08:05:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7187
          1       1.00      0.99      1.00       274

avg / total       1.00      1.00      1.00      7461

('F1 score:', 0.99633699633699624)
('count: ', 43)
(5467, 149210)
[08:05:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7186
          1       1.00      0.99      0.99       275

avg / total       1.00      1.00      1.00      7461

('F1 score:', 0.9926739926739927)
('count: ', 44)
(5467, 149210)
[08:05:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
             precision    r

('count: ', 63)
(5467, 149210)
[08:09:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7185
          1       1.00      0.99      1.00       276

avg / total       1.00      1.00      1.00      7461

('F1 score:', 0.99636363636363634)
('count: ', 64)
(5467, 149210)
[08:09:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7198
          1       1.00      1.00      1.00       263

avg / total       1.00      1.00      1.00      7461

('F1 score:', 0.99809523809523815)
('count: ', 65)
(5467, 149210)
[08:10:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
             precision    

In [14]:
CV_X = pd.concat([sepsis_x_cv, x_cv])
cv_y = pd.concat([sepsis_y_cv, y_cv])
print(classification_report(cv_y, (model.predict(clf1.DMatrix(CV_X.drop(['label'],axis=1), label=cv_y))>0.5).astype(int)))
print('F1 score:', f1_score(cv_y, (model.predict(clf1.DMatrix(CV_X.drop(['label'],axis=1), label=cv_y))>0.5).astype(int)))


             precision    recall  f1-score   support

          0       0.99      0.99      0.99     36299
          1       0.83      0.73      0.78      1367

avg / total       0.98      0.99      0.98     37666

('F1 score:', 0.78067783404752633)
